# Customer Value Segmentation

Mobitel pvt ltd wants to segment their prepaid subscriber base in order to take best action on retention, penetration, elasticity, acquisition and loyalty. The first scheme of the segmentation is the customer value segmentation.

## Problem statement

###### “What are the identifiable customer segments based on their product purchasing behavior for target marketing?”

## Hypothesis generation

We can identify four major product in prepaid bussiness. Revenue of these products are used for customer value segmentation
* VOICE (M2M & M2O) revenue
* IDD revenue
* DATA revenue
* VAS revenue

## Data exploration / transformation

In [1]:
% pylab inline 

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

Populating the interactive namespace from numpy and matplotlib


In [2]:
# reading the sample data
df = pd.read_csv('RevenueStream.csv')

In [5]:
df_backup = df.copy()
df.head(n=2)

,MSISDN,IDD,M2M,M2O,Data_excess,VAS,Data_Addon
0,22690085,0,0,0,0,7.48,0
1,22690327,0,17,0,0,104.72,0


In [6]:
# Deriving the major product revenues
df['VOICE'] = df.M2M + df.M2O
df['DATA'] = df.Data_excess + df.Data_Addon
df['TOTAL'] = df.IDD + df.VOICE + df.DATA + df.VAS

df = df.query('TOTAL >0')
# Removing additional columns
df.drop(['M2M','M2O','Data_Addon','Data_excess'],inplace=True,axis=1)

df.head(n=2)

C:\Users\amalka\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,MSISDN,IDD,VAS,VOICE,DATA,TOTAL
0,22690085,0,7.48,0,0,7.48
1,22690327,0,104.72,17,0,121.72


#### Basic rule based segmentation

In [ ]:
#Customer segment with 0 usage

df['cluster'] ='dormant'

#Voice only customer
df.loc[(df.IDD + df.DATA + df.VAS == 0) & df.TOTAL >0, 'cluster'] = 'voice_only'

#data only base
df.loc[(df.IDD + df.VOICE + df.VAS == 0) & df.TOTAL >0, 'cluster'] = 'data_only'

#idd only base
df.loc[(df.VOICE + df.DATA + df.VAS == 0) & df.TOTAL >0, 'cluster'] = 'idd_only'

#vas only base
df.loc[(df.IDD + df.DATA + df.VOICE == 0) & df.TOTAL >0, 'cluster'] = 'vas_only'

#multi product base
df.loc[((df.TOTAL > df.IDD) & (df.TOTAL > df.DATA) & (df.TOTAL > df.VOICE) & (df.TOTAL >df.VAS)), 'cluster'] = 'multiprod'
multi_prod = df.query('TOTAL > IDD and TOTAL > VAS and TOTAL > IDD and TOTAL > VOICE')


#pd.Series([len(dormant),len(voice_only),len(data_only),len(idd_only),len(vas_only),len(multi_prod)], index=['dormant','Voice_only', 'data_only', 'idd_only', 'vas_only', 'multi_prod'])

In [8]:
multi_prod.head(10)

,MSISDN,IDD,VAS,VOICE,DATA,TOTAL,cluster
1,22690327,0,104.72,17,0,121.72,multiprod
2,22690481,0,149.60,224,0,373.60,multiprod
4,22690558,0,201.96,14,0,215.96,multiprod
5,22690657,0,194.48,167,0,361.48,multiprod
6,22690734,0,206.48,76,12,294.48,multiprod
7,22690756,0,187.00,4,0,191.00,multiprod
8,22690888,0,201.96,65,0,266.96,multiprod
9,22690987,0,209.44,9,0,218.44,multiprod
10,22691042,0,187.00,121,0,308.00,multiprod
11,22691064,0,376.00,299,189,864.00,multiprod


## Clustering multiple product userbase

In [9]:
# Droping IDD IDD user base since it is small
multi_prod_copy = multi_prod.copy()
multi_prod = multi_prod[['TOTAL','VAS','VOICE','DATA']]
# Transforming the absolute usage to propotion value of total use
multi_prod = multi_prod.div(multi_prod["TOTAL"],axis=0)
multi_prod.drop(['TOTAL'],inplace=True,axis=1)

In [10]:
km = KMeans(n_clusters=7).fit(multi_prod)

In [11]:
# cluster representation
multi_prod['cluster'] =  km.labels_
multi_prod_copy['cluster'] =  km.labels_
clustergrp = pd.concat([multi_prod.groupby('cluster').mean().round(2), multi_prod.groupby('cluster')['cluster'].count()], axis=1)
print(clustergrp)

          VAS  VOICE  DATA  cluster
cluster                            
0        0.03   0.91  0.05     5277
1        0.06   0.22  0.72     1456
2        0.84   0.13  0.03     2156
3        0.45   0.53  0.01     1640
4        0.44   0.08  0.48     5343
5        0.30   0.31  0.34     3393
6        0.10   0.61  0.25     3254


In [12]:
df.round(0).to_csv('dfRev.csv',index=True)
multi_prod_copy.to_csv('multi_prod.csv',index=True)